In [15]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics import tsaplots
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [57]:
ne_train = pd.read_csv('ne_train (1).csv')
s_train = pd.read_csv('s_train.csv')
ne_test = pd.read_csv('ne_test.csv')
s_test = pd.read_csv('s_test.csv')

In [58]:
ne_train = pd.merge(ne_train.date, ne_train["covid_19_confirmed_cases"], 
                                left_index=True, right_index=True)
ne_test = pd.merge(ne_test.date, ne_test["covid_19_confirmed_cases"], 
                                left_index=True, right_index=True)

s_train = pd.merge(s_train.date, s_train["covid_19_confirmed_cases"], 
                                left_index=True, right_index=True)
s_test = pd.merge(s_test.date, s_test["covid_19_confirmed_cases"], 
                                left_index=True, right_index=True)

In [59]:
ne_train['cases_shifted'] = ne_train['covid_19_confirmed_cases'].shift(1)
s_train['cases_shifted'] = s_train['covid_19_confirmed_cases'].shift(1)

In [60]:
ne_train.set_index("date", inplace = True)
ne_test.set_index("date", inplace = True)

# ARIMA Model - NE

In [61]:
result = adfuller(ne_train['covid_19_confirmed_cases'])
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')
print('Critical Values:')
for key, value in result[4].items():
    print(f'   {key}: {value}')

ADF Statistic: -2.6006703984315087
p-value: 0.09285878886366933
Critical Values:
   1%: -3.4776006742422374
   5%: -2.882265832283648
   10%: -2.5778219289774156


In [62]:
if result[1] > 0.05:
    ne_train['covid_19_confirmed_cases'] = ne_train['covid_19_confirmed_cases'].diff().dropna()

In [64]:
p = 1 
q = 1 

In [66]:
ne_model = ARIMA(ne_train['cases_shifted'], order=(p, 1, q))
ne_results = ne_model.fit()

/Users/lalla-aichaadouim/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/lalla-aichaadouim/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/lalla-aichaadouim/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/Users/lalla-aichaadouim/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/Users/lalla-aichaadouim/opt/anaconda3

In [68]:
ne_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:          cases_shifted   No. Observations:                  151
Model:                 ARIMA(1, 1, 1)   Log Likelihood               -1299.507
Date:                Fri, 10 Nov 2023   AIC                           2605.015
Time:                        15:58:04   BIC                           2614.047
Sample:                    03-02-2020   HQIC                          2608.684
                         - 07-30-2020                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.3324      0.948     -0.351      0.726      -2.189       1.525
ma.L1          0.3780      0.930      0.406      0.684      -1.445       2.201
sigma2      1.966e+06   1.13e+05     17.320      0.000    1.74e+06    2.19e+06
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):               281.53
Prob(Q):                              0.86   Prob(JB):                         0.00
Heteroskedasticity (H):               0.06   Skew:                             0.53
Prob(H) (two-sided):                  0.00   Kurtosis:                         9.63
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""